# 尼崎市のQAデータ

## 概要

**データセットを以下の2種類のデータとして再構築する**  

`df_qa`: Queryとその回答に必要な参考情報のID
- Query: 質問
- ID: 回答に必要な参考情報のID

`df_ref`:
- ID: 回答に必要な参考情報のID
Reference: 参考情報

## imports

In [56]:
import os
import copy
import pandas as pd
from typing import List

import warnings
warnings.simplefilter('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)
pd.set_option("display.width", 100)
pd.set_option("display.max_colwidth", None)

## 1. [尼崎市のQAデータ](https://github.com/ku-nlp/bert-based-faqir/tree/master)リポジトリをダウンロードしてdownload.shを実行する

`データの取り扱いの参考記事`  
- [Azure AI Searchを使った同義語によるクエリ拡張とその効果
](https://hironsan.hatenablog.com/entry/information-retrieval-with-synonyms)

## 2. 質問と参考情報IDデータの読み込み

In [58]:
# クエリを含む評価データ
df_qa = pd.read_csv("./dataset/04_Amagasaki/bert-based-faqir-master/data/localgovfaq/testset.txt", sep='\t', header=None)
df_qa.columns = ["Query","A","B","C"]
df_qa.shape

(784, 4)

In [59]:
df_qa.head(2)

,Query,A,B,C
0,国民年金の免除申請をしたいのですが、申請に必要な持ち物は何を持っていけば良いですか？,71 87,469,36
1,忍たま乱太郎の施設などはオープンしないのですか,NaN,1770 1769,NaN


**データの説明**
- 2~4列目には正解となるデータのIDがレベル毎に記録されている。レベルはA,B,Cの３種類で以下の詳細を参照。
- **IDは qas/questions_Amagasaki.txt と qas/answers_in_Amagasaki.txtの行数に対応している。**
```
A: 正解を含む.
B: 関連情報を含む.
C: トピックは同じだが、関連情報は含んでいない.
```

## 3. 正解IDの取り出し

In [60]:
def unify_ids(x,include_ids=["A","B","C"]):
    ids = " ".join([str(x[c]) for c in include_ids])
    ids = [x for x in ids.split(" ") if len(x)>0]
    return ids

df_qa[["A","B","C"]] = df_qa[["A","B","C"]].fillna("")

# 3段階のレベルに対応した正解IDを構築
df_qa["ID_ABC"] = df_qa.apply(unify_ids, axis=1,include_ids=(["A","B","C"]))
df_qa["ID_AB"] = df_qa.apply(unify_ids, axis=1,include_ids=(["A","B"]))
df_qa["ID_A"] = df_qa.apply(unify_ids, axis=1,include_ids=(["A"]))

In [65]:
df_qa["ID"] = df_qa["ID_ABC"]

In [66]:
df_qa.head(3)

,Query,A,B,C,ID_ABC,ID_AB,ID_A,ID
0,国民年金の免除申請をしたいのですが、申請に必要な持ち物は何を持っていけば良いですか？,71 87,469,36,"[71, 87, 469, 36]","[71, 87, 469]","[71, 87]","[71, 87, 469, 36]"
1,忍たま乱太郎の施設などはオープンしないのですか,,1770 1769,,"[1770, 1769]","[1770, 1769]",[],"[1770, 1769]"
2,人口が伸び悩む尼崎市ですが、明石市の様な子ども医療費助成を取り入れる計画はありますか？,,,593 375,"[593, 375]",[],[],"[593, 375]"


## 4. 検索対象データ（尼崎市のQAデータ）の読み込み

尼崎市のQAデータは、`df`内のQueryを過去のQ&A(`df_question`と`df_answer`)を検索して参考に回答するタスクを想定している。そのため、`df_question`は不要。

In [62]:
# 検索対象データ（尼崎市のQAデータ）
df_question = pd.read_csv("./dataset/04_Amagasaki/bert-based-faqir-master/data/localgovfaq/qas/questions_in_Amagasaki.txt", index_col=0, sep='\t', header=None)
df_answer = pd.read_csv("./dataset/04_Amagasaki/bert-based-faqir-master/data/localgovfaq/qas/answers_in_Amagasaki.txt", index_col=0, sep='\t', header=None)

print(df_question.shape,df_answer.shape)

# 統合
df_ref = pd.concat([df_question,df_answer],axis=1)
df_ref.columns = ["Query","Reference"]

# 不要な空白を削除
df_ref["Query"] = df_ref["Query"].map(lambda x:x.replace(" ",""))
df_ref["Reference"] = df_ref["Reference"].map(lambda x:x.replace(" ",""))
df_ref = df_ref.reset_index(drop=True)
df_ref["ID"] = list(range(len(df_ref)))
df_ref = df_ref[["ID","Reference"]] # "Query"は不要のためDrop
print(df_ref.shape)

df_ref.head(2)

(1786, 1) (1786, 1)
(1786, 2)


,ID,Reference
0,0,■市には乳幼児（乳児）とその親が集う場として、次のような取組をしています。以下のいずれについて知りたいですか。１．つどいの広場事業、すこやかプラザ運営事業子育て支援ゾーンＰＡＬ。２．保育所子育て支援事業。３．幼稚園すこやか子育て事業。４．子育てサークル活動支援事業。５．子育て学習世代間交流事業。６．双子のための育児教室学級。７．健康教室。
1,1,■地域総合センター今北には、十分な駐車場がございませんので、市バスをご利用ください。ＪＲ沿線からは「立花駅」、阪急沿線からは「塚口駅」「武庫之荘駅」、阪神沿線からは「尼崎駅」「武庫川駅」「出屋敷駅」へお越しいただき、市バスをご利用ください。どちらの駅からいらっしゃいますか？。１．ＪＲ立花駅から（位置的には、南西へ徒歩約１０分です。）。２．阪急塚口駅（南）から。３．阪急武庫之荘駅（南）から。４．阪神尼崎駅（北）から。５．阪神武庫川駅から。６．阪神出屋敷駅（北）から。＜改＞。【関連するＦＡＱ】。地域総合センターについて知りたい。＜改＞。【お問い合わせ】。地域総合センター今北。尼崎市西立花町３丁目１４－１。電話０６－６４１６－５７２９。


In [63]:
# df_ref["Query"].nunique(),
df_ref["Reference"].nunique()

1756

In [64]:
df_qa.head()

,Query,A,B,C,ID_ABC,ID_AB,ID_A
0,国民年金の免除申請をしたいのですが、申請に必要な持ち物は何を持っていけば良いですか？,71 87,469,36,"[71, 87, 469, 36]","[71, 87, 469]","[71, 87]"
1,忍たま乱太郎の施設などはオープンしないのですか,,1770 1769,,"[1770, 1769]","[1770, 1769]",[]
2,人口が伸び悩む尼崎市ですが、明石市の様な子ども医療費助成を取り入れる計画はありますか？,,,593 375,"[593, 375]",[],[]
3,戸籍謄本の発行をして欲しいのですが、代金はいくらかかかりますか？,119 386,,19 1131 883 42,"[119, 386, 19, 1131, 883, 42]","[119, 386]","[119, 386]"
4,原付の登録をしようと思っています。登録にかかる料金と必要な持ち物が知りたいので教えてください。,191,32 73,192,"[191, 32, 73, 192]","[191, 32, 73]",[191]


## 5. 保存

In [69]:
dataset_dir = "./dataset/04_Amagasaki"
os.makedirs(dataset_dir)
df_ref.to_csv("./dataset/04_Amagasaki/Reference.csv")
df_qa.to_csv("./dataset/04_Amagasaki/QA.csv")

In [53]:
# df_ref.shape,df_ref["ID"].nunique()